In [1851]:
def see_file(filename):
    df = pd.read_csv(filename)
    for i in range(len(df)):
        text = df['article'][i]
        print(text)

In [1852]:
import pandas as pd
import re
filename = "D:\\university\\2course\\project\\clear of texts\\dsProject\\outastrophysics_0.csv"

In [1853]:
#огда известна точная концовка
def delete_part_of_text(text, start, end, plusend):
    if_part = text.find(start)
    while if_part != -1:
        end_of_part = text.find(end)
        text = text[:if_part] + text[end_of_part + plusend:]
        if_part = text.find(start)
    return text

In [1854]:
#при поиске ближайшего символа окончания от старта тега
def delete_tag(text, start, end):
    if_link = text.find(start)
    while if_link != -1:
        end_of_link = text[if_link:].find(end)
        #print(text[if_link:end_of_link + 1 + if_link])
        text = text[:if_link] + text[end_of_link + 1 + if_link:]
        if_link = text.find(start)
    return text

In [1855]:
def find_ending(text):
    if text.find('\\begin{references}'):
        text = text[:text.find('\\begin{references}')]

    if text.find('\\end{acknowledgements}'):
        text = text.replace('\\begin{acknowledgements}', '')
        text = text[:text.find('\\end{acknowledgements}')]

    if text.find('\\begin{thebibliography}'):
        text = text[:text.find('\\begin{thebibliography}')]
    return text

In [1856]:
def find_start(text):
    if text.find('\\abstract{') != -1:
        text = text[text.find('\\abstract{'):]
        text = text[text.find('}\n')+2:]

    elif text.find('\\begin{abstract}') != -1:
        text = text[text.find('\\end{abstract}') + 14:]

    if text.find('\\section{Introduction}') != -1:
        text = text[text.find('\\section{Introduction}') + 23:]
    return text

In [1857]:
#для удаления скобок, элементов "окружающих" текст
def side_details(text, start, ending):
    cursive = text.find(start)
    while cursive != -1:
        cursive_end = cursive + text[cursive:].find(ending)
        text = text[:cursive] + text[cursive + len(start) : cursive_end] + text[cursive_end+1:]
        cursive = text.find(start)
    return text

In [1858]:
def clear(text):
    text = find_start(text)

    #поиск окончания основной части
    text = find_ending(text)

    text = delete_tag(text, '\\section{', '}')
    text = delete_tag(text, '\\section {', '}')
    text = delete_tag(text, '\n\subsection', '}')
    text = delete_tag(text, '\n\subsubsection', '}')
    #удаление ссылок на таблицы и рисунки
    text = delete_tag(text, '\\ref{', '}')

    mas = text.split('\n')
    i = 0
    while i < len(mas):
        if i >= 0 and mas[i].strip().startswith('%'):
            #print(mas[i])
            mas.remove(mas[i])
            i -= 1
        if i >= 0 and mas[i].strip().find('\\begin{deluxetable') == 0:
            j = i
            while j < len(mas) and mas[j].strip().find('\\end{deluxetable') != 0:
                j += 1
            #print(mas[i-1:j])
            mas = mas[:i] + mas[j+1:]
            i -= 1
        if i >= 0 and i < len(mas) and mas[i].strip().startswith('\\begin{table'):
            j = i
            while j < len(mas) and (mas[j].strip().find('\\end{table') != 0 and mas[j].strip().find('\\end {table') != 0):
                j += 1
            #print(mas[i-1:j])
            mas = mas[:i] + mas[j+1:]
            i = 0
        i += 1

    text = '\n'.join(mas)

    text = delete_tag(text, '\\begin{figure', '\\end{figure')

    #удаление % TABLE - оставшаяся часть таблиц
    text = text.replace('\n% TABLE\n', '')

    text = delete_tag(text, '\n\label', '}')
    text = side_details(text, '{\\bf', '}')
    text = side_details(text, '{\\underline', '}')
    text = side_details(text, '{\sl', '}')
    text = side_details(text, '{\sc', '}')
    text = text.replace('\\begin{itemize}', '')
    text = text.replace('\\end{itemize}', '')
    text = text.replace('\\medskip', '')
    text = text.replace('\\item ', '')
    mas = text.split('\n')
    i = 0

    while i < len(mas):
        if mas[i].strip().startswith('%') and mas[i].strip().endswith('%') or (mas[i].strip().startswith('%\end{tab')) or ('%%%%%' in mas[i]) or mas[i].strip().startswith('%\\begin{tab') or mas[i].strip().startswith('\\vskip') or ('end{figure') in mas[i].strip() or mas[i].strip().startswith('\\setcounter{table'):
            #print(mas[i])
            mas.remove(mas[i])
            i -= 1
        i += 1
    text = '\n'.join(mas)

    text = delete_tag(text, '\\cite', '}')
    text = text.replace('\\begin{enumerate}', '')
    text = text.replace('\\end{enumerate}', '')
    text = text.replace('\\noindent', '')
    text = delete_tag(text, '\\figcaption', '}\n')
    text = delete_tag(text, '\\resizebox', '}\n')
    text = text.replace('\\acknowledgments', '')
    text = side_details(text, '{\\it', '}')
    text = side_details(text, '\\acknowledgement{', '}')
    text = delete_tag(text, '%___', '_\n')
    text = delete_tag(text, '{\\footnotesize', '}\n')
    text = text.replace('\pagebreak', '')

    if text.find('\\begin{document}') != -1 and len((text[text.find('\\begin{document}'):text.find('\\end{document}')]).split()) < 10:
        text = None

    return text

In [1859]:
def see_file(filename):
    df = pd.read_csv(filename)
    cl = [0] * len(df)
    for i in range(len(df)):
        text = df['Latex'][i]
        text = clear(text)
        cl[i] = (text)
    df['clear'] = cl
    print(df.head())
    print(len(df))
    return(df)


df = see_file(filename)
#print(df.head())
df.to_csv('base_with_clear_texts.csv')

   Unnamed: 0  Unnamed: 0.1  \
0           0             0   
1           1             1   
2           2             2   
3           3             3   
4           4             4   

                                               Title  \
0  Title:\r\nSpectral variability in transonic di...   
1  Title:\r\nThe Nature of the Emission Component...   
2  Title:\r\nConstraints on the Cosmic Equation o...   
3  Title:\r\nModel-independent insights into the ...   
4  Title:\r\nThe First Detection of Cobalt in a D...   

                                          Annotation  \
0  \r\nAbstract: Transonic discs with accretion r...   
1  \r\nAbstract: We present the study of the emis...   
2  \r\nAbstract: In this paper, observational con...   
3  \r\nAbstract: Straightforward physical argumen...   
4  \r\nAbstract: We present the first ever detect...   

                                             PDF  \
0  https://export.arxiv.org/pdf/astro-ph/0104101   
1  https://export.arxiv.org/pdf/ast

In [1860]:
'''text = df['Latex'][612]
print(df['PDF'][612])
#print(text)
with open('text.txt', 'w', encoding='utf-8') as file:
    file.write(text)
text = clear(text)
print(text)'''

"text = df['Latex'][612]\nprint(df['PDF'][612])\n#print(text)\nwith open('text.txt', 'w', encoding='utf-8') as file:\n    file.write(text)\ntext = clear(text)\nprint(text)"